#  Artificial Intelligence Capstone Project on Retail

### Project Task: Week 1

In [ ]:
pip install nbconvert

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.linear_model import LinearRegression,Ridge,ElasticNet,Lasso
from sklearn.metrics import mean_squared_error,mean_absolute_error,accuracy_score,r2_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.decomposition import PCA

from statsmodels.tsa.stattools import adfuller
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [3]:
train = pd.read_csv("/Users/AbiiinSW/Downloads/Project 3-Retail-Datasets/train_data.csv")
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2,2015-06-30,5735,568,1,1,0,0
1,2,2,2015-06-30,9863,877,1,1,0,0
2,3,2,2015-06-30,13261,1072,1,1,0,1
3,4,2,2015-06-30,13106,1488,1,1,0,0
4,5,2,2015-06-30,6635,645,1,1,0,0


In [4]:
test_val= pd.read_csv("/Users/AbiiinSW/Downloads/Project 3-Retail-Datasets/test_data_hidden.csv")
test_val.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
test= pd.read_csv("/Users/AbiiinSW/Downloads/Project 3-Retail-Datasets/test_data.csv")
test.head()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,1,1,0,1
1,2,5,2015-07-31,1,1,0,1
2,3,5,2015-07-31,1,1,0,1
3,4,5,2015-07-31,1,1,0,1
4,5,5,2015-07-31,1,1,0,1


In [6]:
train_1 = train.copy()
test_val_1 = test_val.copy()
test_1 = test.copy()

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982644 entries, 0 to 982643
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Store          982644 non-null  int64 
 1   DayOfWeek      982644 non-null  int64 
 2   Date           982644 non-null  object
 3   Sales          982644 non-null  int64 
 4   Customers      982644 non-null  int64 
 5   Open           982644 non-null  int64 
 6   Promo          982644 non-null  int64 
 7   StateHoliday   982644 non-null  object
 8   SchoolHoliday  982644 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 67.5+ MB


In [8]:
train.isna().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [9]:
test.isna().sum()

Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [10]:
train.DayOfWeek.value_counts()

2    141204
1    140270
4    140270
5    140270
6    140270
7    140270
3    140090
Name: DayOfWeek, dtype: int64

In [11]:
test.DayOfWeek.value_counts()

3    5575
4    5575
5    5575
1    4460
2    4460
6    4460
7    4460
Name: DayOfWeek, dtype: int64

In [12]:
train.Open.value_counts()

1    814204
0    168440
Name: Open, dtype: int64

In [13]:
test.Open.value_counts()

1    30188
0     4377
Name: Open, dtype: int64

In [14]:
train.Promo.value_counts()

0    609059
1    373585
Name: Promo, dtype: int64

In [15]:
test.Promo.value_counts()

0    20070
1    14495
Name: Promo, dtype: int64

In [16]:
train.StateHoliday.unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [17]:
test.StateHoliday.value_counts()

0    34565
Name: StateHoliday, dtype: int64

In [18]:
train.SchoolHoliday.value_counts()

0    813700
1    168944
Name: SchoolHoliday, dtype: int64

In [19]:
test.SchoolHoliday.value_counts()

0    21788
1    12777
Name: SchoolHoliday, dtype: int64

In [20]:
train.Date.unique()

array(['2015-06-30', '2015-06-29', '2015-06-28', '2015-06-27',
       '2015-06-26', '2015-06-25', '2015-06-24', '2015-06-23',
       '2015-06-22', '2015-06-21', '2015-06-20', '2015-06-19',
       '2015-06-18', '2015-06-17', '2015-06-16', '2015-06-15',
       '2015-06-14', '2015-06-13', '2015-06-12', '2015-06-11',
       '2015-06-10', '2015-06-09', '2015-06-08', '2015-06-07',
       '2015-06-06', '2015-06-05', '2015-06-04', '2015-06-03',
       '2015-06-02', '2015-06-01', '2015-05-31', '2015-05-30',
       '2015-05-29', '2015-05-28', '2015-05-27', '2015-05-26',
       '2015-05-25', '2015-05-24', '2015-05-23', '2015-05-22',
       '2015-05-21', '2015-05-20', '2015-05-19', '2015-05-18',
       '2015-05-17', '2015-05-16', '2015-05-15', '2015-05-14',
       '2015-05-13', '2015-05-12', '2015-05-11', '2015-05-10',
       '2015-05-09', '2015-05-08', '2015-05-07', '2015-05-06',
       '2015-05-05', '2015-05-04', '2015-05-03', '2015-05-02',
       '2015-05-01', '2015-04-30', '2015-04-29', '2015-

In [31]:
train['Date'].value_counts()

2014-05-20    1115
2013-07-28    1115
2013-01-21    1115
2013-06-26    1115
2014-01-02    1115
              ... 
2014-12-30     935
2014-07-30     935
2014-09-01     935
2014-10-05     935
2014-07-07     935
Name: Date, Length: 911, dtype: int64

In [32]:
test_val.sort_values(['Store'],inplace=True)
test.sort_values(['Store'],inplace=True)
combi = train.append(test_val , ignore_index=True)
print(combi.shape)
combi =combi.append(test , ignore_index=True)
print(combi.shape)
combi['year']=pd.to_datetime(combi['Date'],format='%Y-%m-%d').dt.year 
combi['month']=pd.to_datetime(combi['Date'],format='%Y-%m-%d').dt.month 
combi['day']=pd.to_datetime(combi['Date'],format='%Y-%m-%d').dt.day
combi['year'] = combi.year.replace({2013 : 0, 2014 : 1 , 2015 : 2 })
combi['StateHoliday'] = combi.StateHoliday.replace({'0' : 0, 'a' : 1 , 'b' : 2 ,'c' : 3})
combi.head()

(1017209, 9)
(1051774, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day
0,1,2,2015-06-30,5735.0,568.0,1,1,0,0,2,6,30
1,2,2,2015-06-30,9863.0,877.0,1,1,0,0,2,6,30
2,3,2,2015-06-30,13261.0,1072.0,1,1,0,1,2,6,30
3,4,2,2015-06-30,13106.0,1488.0,1,1,0,0,2,6,30
4,5,2,2015-06-30,6635.0,645.0,1,1,0,0,2,6,30


In [33]:
combi1= pd.get_dummies(combi,columns=['DayOfWeek', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 'year','Store','day','month'],drop_first=True)
# combi1=pd.read_csv('combi1.csv')
# combi1.drop(['Unnamed: 0'],axis=True,inplace=True)
combi1.head()

,Date,Sales,Customers,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Open_1,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2015-06-30,5735.0,568.0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,2015-06-30,9863.0,877.0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,2015-06-30,13261.0,1072.0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,2015-06-30,13106.0,1488.0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,2015-06-30,6635.0,645.0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [34]:
combi2= pd.get_dummies(combi,columns=['DayOfWeek', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 'year','day','month'],drop_first=True)
combi2.head()

,Store,Date,Sales,Customers,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,2015-06-30,5735.0,568.0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2,2015-06-30,9863.0,877.0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,3,2015-06-30,13261.0,1072.0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,4,2015-06-30,13106.0,1488.0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,2015-06-30,6635.0,645.0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [35]:
train.shape,test_val.shape,test.shape

((982644, 9), (34565, 9), (34565, 7))

In [36]:
train1 = combi1.iloc[:982644].reset_index(drop=True)
test_val1 = combi1.iloc[982644:1017209].reset_index(drop=True)
test1 = combi1.iloc[1017209:].reset_index(drop=True)
train1.shape,test_val1.shape,test1.shape

((982644, 1172), (34565, 1172), (34565, 1172))

In [37]:
train2 = combi2.iloc[:982644].reset_index(drop=True)
test_val2 = combi2.iloc[982644:1017209].reset_index(drop=True)
test2 = combi2.iloc[1017209:].reset_index(drop=True)
train2.shape,test_val2.shape,test2.shape

((982644, 59), (34565, 59), (34565, 59))

In [38]:
train.corr()['Sales']

Store            0.005338
DayOfWeek       -0.461249
Sales            1.000000
Customers        0.895700
Open             0.679248
Promo            0.451383
SchoolHoliday    0.076141
Name: Sales, dtype: float64

In [39]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
x=train._get_numeric_data()
vif=pd.DataFrame()
vif["VIF"]=[variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
vif["features"]=x.columns
vif

,VIF,features
0,3.541150,Store
1,2.773128,DayOfWeek
2,21.956526,Sales
3,15.190168,Customers
4,8.210847,Open
5,2.154974,Promo
6,1.207436,SchoolHoliday


#### Linear Regression with STORE as feature

In [40]:
Y_train = train1['Sales']
Y_val = test_val1['Sales']

In [41]:
X_train = train1.drop(['Sales','Date','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Customers'],axis=1).values
lr_1 = LinearRegression()
lr_1.fit(X_train,Y_train)
Y_pred1 = lr_1.predict(X_val)
print('MSE',np.sqrt(mean_squared_error(Y_pred1,Y_val)))
print('MAE',mean_absolute_error(Y_pred1,Y_val))
print('train model score',lr_1.score(X_train,Y_train))
print('test model score',lr_1.score(X_val,Y_val))

MSE 1428.9181706826778
MAE 1051.5557239043487
train model score 0.8365645595889427
test model score 0.8430035399815972


#### Linear Regression without STORE as feature

In [42]:
X_train1 = train2.drop(['Sales','Date','Customers'],axis=1).values
X_val1 = test_val2.drop(['Sales','Date','Customers'],axis=1).values
lr_2 = LinearRegression()
lr_2.fit(X_train1,Y_train)
Y_pred2 = lr_2.predict(X_val1)
print('MSE',np.sqrt(mean_squared_error(Y_pred2,Y_val)))
print('MAE',mean_absolute_error(Y_pred2,Y_val))
print('train model score',lr_2.score(X_train1,Y_train))
print('test model score',lr_2.score(X_val1,Y_val))

MSE 2520.0716734481693
MAE 1731.704737951633
train model score 0.5646267338030363
test model score 0.5116840301951011


#### Linear Regression - Separate model for each STORE

In [43]:
Y_pred3=np.zeros(test_val.shape[0])

train_store = train2.groupby(['Store'])
test_store = test_val2.groupby(['Store'])

for i in range(1,1116):
    a = train_store.get_group(i)
    b = test_store.get_group(i)
    X_train = a.drop(['Sales','Date','Store','Customers'],axis=1).values
    X_val = b.drop(['Sales','Date','Store','Customers'],axis=1).values
    Y_train = a['Sales']
    #Y_val = b['Sales']
    lr = LinearRegression()
    lr.fit(X_train,Y_train)
    pred = lr.predict(X_val)
    i=0
    for j in b.index:
        Y_pred3[j]=pred[i]
        i+=1
print('MSE',np.sqrt(mean_squared_error(Y_pred3,Y_val)))
print('MAE',mean_absolute_error(Y_pred3,Y_val))

MSE 617791598632999.0
MAE 14098079978199.238


So from the above 3 models we can conclude that the model perform better with 'Store' as feature.
Also the average of all the separate model based on Store Id is the worst model.

#### Average Ensemble Model of first and second model

In [44]:
final_pred=(Y_pred1+Y_pred2)/2
print('MSE',np.sqrt(mean_squared_error(final_pred,Y_val)))
print('MAE',mean_absolute_error(final_pred,Y_val))

MSE 1786.5727235271445
MAE 1295.536559435401


#### Weighted Average Ensemble Model of first and second model

In [45]:
final_pred=Y_pred1*0.7+Y_pred2*0.3
print('MSE',np.sqrt(mean_squared_error(final_pred,Y_val)))
print('MAE',mean_absolute_error(final_pred,Y_val))

MSE 1578.214817787205
MAE 1163.7769636251792


In [48]:
import warnings
warnings.filterwarnings('ignore')

#### Regularization of 1st Model

In [49]:
X_train = train1.drop(['Sales','Date','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Customers'],axis=1).values
rr =Ridge(alpha=10)
rr.fit(X_train,Y_train)
Y_pred1 = rr.predict(X_val)
print('MSE',np.sqrt(mean_squared_error(Y_pred1,Y_val)))
print('MAE',mean_absolute_error(Y_pred1,Y_val))
print('train model score',rr.score(X_train,Y_train))
print('test model score',rr.score(X_val,Y_val))

ValueError: Found input variables with inconsistent numbers of samples: [982644, 911]

Regualrization technique is not enhancing the performance.

### Project Task: Week 2
 

In [50]:
train=train[train.Open==1]
shape1=train.shape[0]
print(train.shape[0])
combi = train.append(test_val , ignore_index=True,sort=False)
shape2=combi.shape[0]
print(combi.shape)
combi =combi.append(test , ignore_index=True,sort=False)
print(combi.shape)
combi['year']=pd.to_datetime(combi['Date'],format='%Y-%m-%d').dt.year 
combi['month']=pd.to_datetime(combi['Date'],format='%Y-%m-%d').dt.month 
combi['day']=pd.to_datetime(combi['Date'],format='%Y-%m-%d').dt.day
combi['year'] = combi.year.replace({2013 : 0, 2014 : 1 , 2015 : 2 })
combi['StateHoliday'] = combi.StateHoliday.replace({'0' : 0, 'a' : 1 , 'b' : 2 ,'c' : 3})
#with Store Id as features
combi1= pd.get_dummies(combi,columns=['DayOfWeek', 'Promo','StateHoliday', 'SchoolHoliday', 'year','Store','day','month'],drop_first=True)
#without Store Id as features
combi2= pd.get_dummies(combi,columns=['DayOfWeek', 'Promo','StateHoliday', 'SchoolHoliday', 'year','day','month'],drop_first=True)

print(train.shape,test_val.shape,test.shape)

train1 = combi1.iloc[:shape1].reset_index(drop=True)
test_val1 = combi1.iloc[shape1:shape2].reset_index(drop=True)
test1 = combi1.iloc[shape2:].reset_index(drop=True)
print(train1.shape,test_val1.shape,test1.shape)

train2 = combi2.iloc[:shape1].reset_index(drop=True)
test_val2 = combi2.iloc[shape1:shape2].reset_index(drop=True)
test2 = combi2.iloc[shape2:].reset_index(drop=True)
print(train2.shape,test_val2.shape,test2.shape)

814204
(848769, 9)
(883334, 9)
(814204, 9) (34565, 9) (34565, 7)
(814204, 1172) (34565, 1172) (34565, 1172)
(814204, 59) (34565, 59) (34565, 59)


In [51]:
Y_train = train1['Sales']
Y_val = test_val1['Sales']

#### Model1

In [52]:

X_train = train1.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Open','Customers'],axis=1).values

lr = LinearRegression()
lr.fit(X_train,Y_train)
pred1 = lr.predict(X_val)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred1[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred1,Y_val)))
print('MAE',mean_absolute_error(pred1,Y_val))

# MSE 1428.9181706827264
# MAE 1051.555723904386

MSE 3519.2466804063965
MAE 2207.5942264072805


#### Model2

In [53]:
X_train1 = train2.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val1 = test_val2.drop(['Sales','Date','Open','Customers'],axis=1).values

lr = LinearRegression()
lr.fit(X_train1,Y_train)
pred2 = lr.predict(X_val1)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred2[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred2,Y_val)))
print('MAE',mean_absolute_error(pred2,Y_val))

# MSE 2520.0716734481657
# MAE 1731.704737951524

MSE 4372.392117586252
MAE 3168.944605518081


#### Model3

In [54]:
pred3=np.zeros(test_val.shape[0])

train_store = train2.groupby(['Store'])
test_store = test_val2.groupby(['Store'])

for i in range(1,1116):
    a = train_store.get_group(i)
    b = test_store.get_group(i)
    X_train = a.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    X_val = b.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    Y_train = a['Sales']
    lr = LinearRegression()
    lr.fit(X_train,Y_train)
    pred = lr.predict(X_val)
    i=0
    ind=b[b['Open']==0].index
    for j in b.index:
        if(j in ind):
            pred3[j]=0
        else:
            pred3[j]=pred[i]
        i+=1
print('MSE',np.sqrt(mean_squared_error(pred3,Y_val)))
print('MAE',mean_absolute_error(pred3,Y_val))

# MSE 2886004774448802.0
# MAE 65858982569725.75

MSE 1013.9379024007219
MAE 669.8719998788438


#### Regularization of Model 3

From the above model,we can see the performance has increased due to data cleaning except in 2nd model which remains almost same. In this case third model has outperformed which was earlier worst model.

In [55]:
train_store = train2.groupby(['Store'])
test_store = test_val2.groupby(['Store'])

for i in range(1,1116):
    a = train_store.get_group(i)
    b = test_store.get_group(i)
    X_train = a.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    X_val = b.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    Y_train = a['Sales']
    lr = Ridge(alpha=20)
    lr.fit(X_train,Y_train)
    pred = lr.predict(X_val)
    i=0
    ind=b[b['Open']==0].index
    for j in b.index:
        if(j in ind):
            pred3[j]=0
        else:
            pred3[j]=pred[i]
        i+=1
print('MSE',np.sqrt(mean_squared_error(pred3,Y_val)))
print('MAE',mean_absolute_error(pred3,Y_val))


MSE 930.9742188387742
MAE 629.3727064444969


Only 3rd model's performance is increasing with regularization 

model3:                 MSE 1014.9293535430203         MAE 670.5513943441184

after reegularization:  MSE 930.9742188387742          MAE 629.3727064444969

#### Random Forest Regression 

In [56]:
#With Store as Feature
X_train = train1.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Open','Customers'],axis=1).values

clf = RandomForestRegressor(n_estimators=500,max_features='sqrt',max_depth=6,random_state=0,n_jobs=7)
clf.fit(X_train,Y_train)
pred1 = clf.predict(X_val)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred1[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred1,Y_val)))
print('MAE',mean_absolute_error(pred1,Y_val))


ValueError: Found input variables with inconsistent numbers of samples: [814204, 754]

In [ ]:
#Without Store as Feature
X_train = train2.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val = test_val2.drop(['Sales','Date','Open','Customers'],axis=1).values
clf = RandomForestRegressor(n_estimators=500,max_features='sqrt',max_depth=6,random_state=0,n_jobs=7)
clf.fit(X_train,Y_train)
pred1 = clf.predict(X_val)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred1[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred1,Y_val)))
print('MAE',mean_absolute_error(pred1,Y_val))

In [ ]:
#Separate model for each Store
pred3=np.zeros(test_val.shape[0])

train_store = train2.groupby(['Store'])
test_store = test_val2.groupby(['Store'])

for i in range(1,1116):
    a = train_store.get_group(i)
    b = test_store.get_group(i)
    X_train = a.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    X_val = b.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    Y_train = a['Sales']
    clf = RandomForestRegressor(n_estimators=500,max_features='sqrt',max_depth=6,random_state=0,n_jobs=7)
    clf.fit(X_train,Y_train)
    pred = clf.predict(X_val)
    i=0
    ind=b[b['Open']==0].index
    for j in b.index:
        if(j in ind):
            pred3[j]=0
        else:
            pred3[j]=pred[i]
        i+=1
print('MSE',np.sqrt(mean_squared_error(pred3,Y_val)))
print('MAE',mean_absolute_error(pred3,Y_val))

#### PCA

In [ ]:
X_train = train1.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Open','Customers'],axis=1).values
pca = PCA().fit(X_train)
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Explained Variance')
plt.show()
# Cumulative Variance explains
# var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
# print(var1.shape)
# print(var1)

In [ ]:
X_train = train1.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Open','Customers'],axis=1).values
Y_train = train1['Sales']
Y_val = test_val1['Sales']

pca = PCA(n_components=50)  
X_train = pca.fit_transform(X_train)  
X_val= pca.transform(X_val) 

clf = RandomForestRegressor(n_estimators=500,max_features='sqrt',max_depth=6,random_state=0,n_jobs=7)
clf.fit(X_train,Y_train)
pred1 = clf.predict(X_val)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred1[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred1,Y_val)))
print('MAE',mean_absolute_error(pred1,Y_val))

#### XGBRegressor

In [ ]:
#With Store as Feature
X_train = train1.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Open','Customers'],axis=1).values

clf = XGBRegressor(n_estimators=500, learning_rate=0.5,max_depth=6,random_state=0,n_jobs=-1)
clf.fit(X_train,Y_train)
pred1 = clf.predict(X_val)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred1[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred1,Y_val)))
print('MAE',mean_absolute_error(pred1,Y_val))


In [ ]:
#Without Store as Feature
X_train = train2.drop(['Sales','Date','Open','Customers'],axis=1).values
X_val = test_val2.drop(['Sales','Date','Open','Customers'],axis=1).values
clf = XGBRegressor(n_estimators=500, learning_rate=0.5,max_depth=6,random_state=0,n_jobs=-1)
clf.fit(X_train,Y_train)
pred1 = clf.predict(X_val)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred1[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred1,Y_val)))
print('MAE',mean_absolute_error(pred1,Y_val))

In [ ]:
#Separate model for each Store
pred3=np.zeros(test_val.shape[0])

train_store = train2.groupby(['Store'])
test_store = test_val2.groupby(['Store'])

for i in range(1,1116):
    a = train_store.get_group(i)
    b = test_store.get_group(i)
    X_train = a.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    X_val = b.drop(['Sales','Date','Store','Customers','Open'],axis=1).values
    Y_train = a['Sales']
    clf = XGBRegressor(n_estimators=500, learning_rate=0.5,max_depth=6,random_state=0,n_jobs=-1)
    clf.fit(X_train,Y_train)
    pred = clf.predict(X_val)
    i=0
    ind=b[b['Open']==0].index
    for j in b.index:
        if(j in ind):
            pred3[j]=0
        else:
            pred3[j]=pred[i]
        i+=1
print('MSE',np.sqrt(mean_squared_error(pred3,Y_val)))
print('MAE',mean_absolute_error(pred3,Y_val))

In [ ]:
X_train.shape,Y_train.shape

In [ ]:
X_train = train1.drop(['Sales','Date','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Customers'],axis=1).values
Y_train = train1['Sales']
Y_val = test_val1['Sales']
pca = PCA(n_components=50) 
X_train = pca.fit_transform(X_train)  
X_val= pca.transform(X_val) 

clf = XGBRegressor(n_estimators=500, learning_rate=0.1,max_depth=6,random_state=0,n_jobs=-1,objective='reg:linear', 
                   booster='gbtree')
clf.fit(X_train,Y_train)
pred1 = clf.predict(X_val)

ind=test_val[test_val.Open==0].index
for i in ind:
    pred1[i] = 0
    
print('MSE',np.sqrt(mean_squared_error(pred1,Y_val)))
print('MAE',mean_absolute_error(pred1,Y_val))

#### Time-series model

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
Train = pd.read_csv("train_data.csv", parse_dates=['Date'], index_col='Date',date_parser=dateparse)
Test_val = pd.read_csv("test_data_hidden.csv", parse_dates=['Date'], index_col='Date',date_parser=dateparse)
Train=Train[['Store','Sales','Open','DayOfWeek']]
Test_val=Test_val[['Store','Sales','Open','DayOfWeek']]
print ('\n Parsed Data:')
Train.sort_values(['Date'],axis=0,inplace=True)
Test_val.sort_values(['Date'],axis=0,inplace=True)
print (Train.head())

#### Store 1

In [ ]:
store1=Train[Train.Store==1]
test_store1=Test_val[Test_val.Store==1]

In [ ]:
sns.boxplot(x="DayOfWeek", y="Sales", data=store1)

Monday=1, Sunday=7.

Here we can find on Sunday stores are closed. Monday has little larger sales, Thurdays has little smaller. There's a few outliers on all days(except Sunday) but it is less on Weekdays(1,3)

In [ ]:
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(store1['Sales'], model='additive',freq=365)
fig = decomposition.plot()
plt.show()

In [ ]:
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    
    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    

In [ ]:
test_stationarity(store1['Sales'])

The smaller p-value, the more likely it's stationary. Here our p-value is 0.000415. It's actually good, but as we just visually found a little downward trend, we want to be more strict, i.e. if the p value further decreases, this series would be  more likely to be stationary.
To get a stationary data, there's many techiniques. We can use log, differencing etc..

In [ ]:
first_diff = store1['Sales'] - store1['Sales'].shift(1)
first_diff = first_diff.dropna(inplace = False)
test_stationarity(first_diff)

After differencing, the p-value is extremely small. Thus this series is very likely to be stationary.

In [ ]:
#AR model
ar_mod = ARIMA(store1.Sales, (9,1,0),freq='D')
res=ar_mod.fit(disp=False)
Y_pred = res.forecast(steps=31)[0]
print('MSE',np.sqrt(mean_squared_error(Y_pred,test_store1.Sales)))
print('MAE',mean_absolute_error(Y_pred,test_store1.Sales))

In [ ]:
#MA model
ma_mod = ARIMA(store1.Sales, (0,1,1),freq='D')
res=ma_mod.fit(disp=False)
Y_pred = res.forecast(steps=31)[0]
print('MSE',np.sqrt(mean_squared_error(Y_pred,test_store1.Sales)))
print('MAE',mean_absolute_error(Y_pred,test_store1.Sales))

In [ ]:
#ARIMA model
arima_mod = ARIMA(store1.Sales, (9,1,9),freq='D')
res=arima_mod.fit(disp=False)
Y_pred = res.forecast(steps=31)[0]
print('MSE',np.sqrt(mean_squared_error(Y_pred,test_store1.Sales)))
print('MAE',mean_absolute_error(Y_pred,test_store1.Sales))
store1['pred']=Y_pred

## Project Task: Week 3

#### Implementing Neural Networks:


#### LSTM for store1

In [ ]:
train_store1 = store1.iloc[:, 1:2].values
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
train_store1 = sc.fit_transform(train_store1)

X_train = []
Y_train = []
for i in range(30, 911):
    X_train.append(train_store1[i-30:i, 0])
    Y_train.append(train_store1[i, 0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units = 30, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(LSTM(units = 70, return_sequences = True))
regressor.add(LSTM(units = 50))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(X_train, Y_train, epochs = 100, batch_size = 64, shuffle=False)


In [ ]:
total_data = pd.concat((store1['Sales'], test_store1['Sales']), axis = 0)
inputs = total_data[len(total_data) - len(test_store1) - 30:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(30, 61):
    X_test.append(inputs[i-30:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
pred = regressor.predict(X_test)
pred= sc.inverse_transform(pred)
print(np.sqrt(mean_squared_error(pred,test_store1.Sales)))
print(mean_absolute_error(pred,test_store1.Sales)) 


In [ ]:
# Visualising the results
plt.plot(test_store1.Sales, color = 'red', label = 'Actual Sales')
plt.plot(pred, color = 'blue', label = 'Predicted Sales')
plt.title('Sales Prediction')
plt.xlabel('Time')
plt.ylabel('Sale')
plt.legend()
plt.show()

#### Applying ANN:


In [ ]:
#Model1
X_train = train2.drop(['Sales','Date','Customers'],axis=1).values
X_val = test_val2.drop(['Sales','Date','Customers'],axis=1).values
Y_train = pd.DataFrame(train2['Sales'])
Y_val = test_val2['Sales']
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
Y_train = sc.fit_transform(Y_train)

model = Sequential()
model.add(Dense(100, activation='relu', input_dim = X_train.shape[1]))
#model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(50, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(1,activation='linear',kernel_initializer='normal') )
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, epochs=10, batch_size=64,shuffle=False,verbose=0)
Y_pred = model.predict(X_val, batch_size=64,verbose=0)
Y_pred= sc.inverse_transform(Y_pred)
print('MSE',np.sqrt(mean_squared_error(Y_pred,Y_val)))
print('MAE',mean_absolute_error(Y_pred,Y_val))
# MSE 2515.353601819651
#MAE 1676.8835278851793

In [ ]:
#model2
X_train = train1.drop(['Sales','Date','Customers'],axis=1).values
X_val = test_val1.drop(['Sales','Date','Customers'],axis=1).values
Y_train = pd.DataFrame(train1['Sales'])
Y_val = test_val1['Sales']
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
Y_train = sc.fit_transform(Y_train)

model = Sequential()
model.add(Dense(100, activation='relu', input_dim = X_train.shape[1]))
#model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='linear') )
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, epochs=10, batch_size=64,shuffle=False,verbose=0)
Y_pred = model.predict(X_val, batch_size=64,verbose=0)
Y_pred= sc.inverse_transform(Y_pred)


print('MSE',np.sqrt(mean_squared_error(Y_pred,Y_val)))
print('MAE',mean_absolute_error(Y_pred,Y_val))